# Google Review Ratings - Clustering Analizi

## 📊 Proje Özeti

Bu notebook, **Google Travel Review Ratings** verisi üzerinde kapsamlı bir **clustering (kümeleme) analizi** gerçekleştirmektedir. Analiz, kullanıcıların farklı seyahat kategorilerindeki değerlendirme davranışlarını inceleyerek benzer kullanıcıları gruplara ayırmayı amaçlamaktadır.

**Veri Kaynağı:** [UCI Machine Learning Repository - Travel Review Ratings](https://archive.ics.uci.edu/dataset/485/tarvel+review+ratings)

---

## 📋 Veri Seti Hakkında

### Genel Bilgiler
- **Toplam Kullanıcı Sayısı:** 5,456 kullanıcı
- **Kategori Sayısı:** 24 farklı seyahat kategorisi
- **Puanlama Sistemi:** 0-5 arası (0 = değerlendirme yok, 1-5 = puanlama)
- **Veri Tipi:** Kullanıcı bazlı değerlendirme matrisi

### Analiz Yöntemleri
Bu notebook'ta aşağıdaki clustering algoritmaları uygulanacaktır:
1. **K-Means Clustering**
2. **Hierarchical Clustering (Agglomerative)**
3. **DBSCAN**
4. **PCA ile Boyut İndirgeme ve Görselleştirme**


## 📦 1. Kütüphanelerin İçe Aktarılması


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

# Görselleştirme ayarları
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✅ Kütüphaneler başarıyla yüklendi!")


✅ Kütüphaneler başarıyla yüklendi!


## 📥 2. Veri Yükleme ve İnceleme


In [ ]:
# Veri yükleme
df = pd.read_csv('data/google_review_ratings.csv')

print("📊 Veri Seti Bilgileri:")
print("=" * 50)
print(f"Satır Sayısı: {df.shape[0]}")
print(f"Sütun Sayısı: {df.shape[1]}")
print(f"\nİlk 5 Satır:")
print(df.head())
print(f"\nVeri Tipleri:")
print(df.dtypes)
print(f"\nEksik Değerler:")
print(df.isnull().sum().sum())
print(f"\nTemel İstatistikler:")
print(df.describe())


📊 Veri Seti Bilgileri:
Satır Sayısı: 5456
Sütun Sayısı: 26

İlk 5 Satır:
     User  Category 1  Category 2  Category 3  Category 4  Category 5  \
0  User 1         0.0         0.0        3.63        3.65         5.0   
1  User 2         0.0         0.0        3.63        3.65         5.0   
2  User 3         0.0         0.0        3.63        3.63         5.0   
3  User 4         0.0         0.5        3.63        3.63         5.0   
4  User 5         0.0         0.0        3.63        3.63         5.0   

   Category 6  Category 7  Category 8  Category 9  ...  Category 16  \
0        2.92         5.0        2.35        2.33  ...         0.59   
1        2.92         5.0        2.64        2.33  ...         0.59   
2        2.92         5.0        2.64        2.33  ...         0.59   
3        2.92         5.0        2.35        2.33  ...         0.59   
4        2.92         5.0        2.64        2.33  ...         0.59   

  Category 17  Category 18  Category 19  Category 20  Categor

## 🔧 3. Veri Ön İşleme


In [ ]:
# User sütununu index olarak ayarla
df.set_index('User', inplace=True)

# Kategori sütunlarını al
category_columns = [col for col in df.columns if 'Category' in col]
print(f"Toplam kategori sayısı: {len(category_columns)}")

# Eksik değerleri kontrol et
print(f"\nEksik değer sayısı: {df[category_columns].isnull().sum().sum()}")

# 0 değerlerini (değerlendirme yapılmamış) NaN olarak işaretle (opsiyonel)
# df[category_columns] = df[category_columns].replace(0, np.nan)

# Veri setini hazırla
X = df[category_columns].copy()

# Eksik değerleri ortalama ile doldur
imputer = SimpleImputer(strategy='mean')
X_imputed = pd.DataFrame(
    imputer.fit_transform(X),
    columns=X.columns,
    index=X.index
)

print(f"\n✅ Veri ön işleme tamamlandı!")
print(f"Veri boyutu: {X_imputed.shape}")


Toplam kategori sayısı: 24

Eksik değer sayısı: 2


ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: '2\t2.'

: 

## 📈 4. Veri Görselleştirme ve Keşifsel Analiz


In [ ]:
# Kategorilerin ortalama puanları
category_means = X_imputed.mean().sort_values(ascending=False)

plt.figure(figsize=(14, 8))
category_means.plot(kind='barh', color='steelblue')
plt.title('Kategorilerin Ortalama Puanları', fontsize=16, fontweight='bold')
plt.xlabel('Ortalama Puan', fontsize=12)
plt.ylabel('Kategori', fontsize=12)
plt.tight_layout()
plt.show()

print("Kategorilerin Ortalama Puanları:")
print(category_means.round(2))


In [ ]:
# Korelasyon matrisi
plt.figure(figsize=(16, 12))
correlation_matrix = X_imputed.corr()
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Kategoriler Arası Korelasyon Matrisi', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()


## 🎯 5. Veri Standardizasyonu


In [ ]:
# Veriyi standardize et
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)
X_scaled_df = pd.DataFrame(X_scaled, columns=X_imputed.columns, index=X_imputed.index)

print("✅ Veri standardizasyonu tamamlandı!")
print(f"Standardize edilmiş veri boyutu: {X_scaled_df.shape}")
print(f"\nStandardize edilmiş veri istatistikleri:")
print(X_scaled_df.describe().round(2))


## 🔍 6. Optimal Küme Sayısının Belirlenmesi (Elbow Method & Silhouette Score)


In [ ]:
# Elbow Method ve Silhouette Score ile optimal küme sayısını bul
inertias = []
silhouette_scores = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))

# Görselleştirme
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Elbow Method
ax1.plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('Küme Sayısı (k)', fontsize=12)
ax1.set_ylabel('Inertia (WCSS)', fontsize=12)
ax1.set_title('Elbow Method - Optimal Küme Sayısı', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Silhouette Score
ax2.plot(K_range, silhouette_scores, 'ro-', linewidth=2, markersize=8)
ax2.set_xlabel('Küme Sayısı (k)', fontsize=12)
ax2.set_ylabel('Silhouette Score', fontsize=12)
ax2.set_title('Silhouette Score - Optimal Küme Sayısı', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# En iyi k değerini bul
best_k = K_range[np.argmax(silhouette_scores)]
print(f"✅ En yüksek Silhouette Score: {max(silhouette_scores):.4f} (k={best_k})")
print(f"\nTüm Silhouette Skorları:")
for k, score in zip(K_range, silhouette_scores):
    print(f"  k={k}: {score:.4f}")


## 🎨 7. K-Means Clustering


In [ ]:
# Optimal küme sayısı ile K-Means
optimal_k = best_k
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
kmeans_labels = kmeans.fit_predict(X_scaled)

# Sonuçları DataFrame'e ekle
results_df = X_imputed.copy()
results_df['KMeans_Cluster'] = kmeans_labels

print(f"✅ K-Means clustering tamamlandı! (k={optimal_k})")
print(f"\nKüme Dağılımı:")
print(results_df['KMeans_Cluster'].value_counts().sort_index())

# Metrikleri hesapla
silhouette_kmeans = silhouette_score(X_scaled, kmeans_labels)
davies_bouldin_kmeans = davies_bouldin_score(X_scaled, kmeans_labels)
calinski_harabasz_kmeans = calinski_harabasz_score(X_scaled, kmeans_labels)

print(f"\n📊 K-Means Metrikleri:")
print(f"  Silhouette Score: {silhouette_kmeans:.4f}")
print(f"  Davies-Bouldin Index: {davies_bouldin_kmeans:.4f}")
print(f"  Calinski-Harabasz Score: {calinski_harabasz_kmeans:.4f}")


In [ ]:
# Kümelerin özelliklerini analiz et
cluster_means = results_df.groupby('KMeans_Cluster')[category_columns].mean()

plt.figure(figsize=(16, 8))
cluster_means.T.plot(kind='bar', figsize=(16, 8))
plt.title('Kümelerin Kategori Ortalamaları (K-Means)', fontsize=16, fontweight='bold')
plt.xlabel('Kategori', fontsize=12)
plt.ylabel('Ortalama Puan', fontsize=12)
plt.legend(title='Küme', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print("Kümelerin Kategori Ortalamaları:")
print(cluster_means.round(2))
